In [1]:
import openai, json, pandas as pd, os


In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
  """
  wrapper for the API to save the prompt and the result
  """
  props = {
  "engine":"davinci",
  "temperature":0,
  "max_tokens":250,
  "stop":"\n\n",
  **kwargs
  }

  r = openai.Completion.create(prompt=prompt, **props)["choices"][0]["text"].strip()
  return r
     


Making sure this works

In [4]:
query("q: what is 1+1?\na:", stop="\n")


'2'

Let's go and start creating some frew shots from the 

In [5]:
def createTargetExample(row):
  return f"""{{
    "Background": "{row.context}",
    "Claim": "{row.hypothesis}",
    "Part of Claim in Background":"""

In [6]:
def evaluateAnswer(res):
    if "entailment" in res.lower()[-30:]:
        return 'e'
    elif "contradiction" in res.lower()[-30:]:
        return 'c'
    return 'n'

In [7]:
basePrompt = """You are an expert analyst evaluating where claims are contradictions, entailments, or neutral with respect to a background. You are given a claim and a background. You must decide whether the claim is a contradiction, entailment, or neutral with respect to the background. You can use the claim and background.
{
"Background": "The others gave in very soon , and longed to be friends , for now there was no Daisy to pet and cook for them ; no Nan to amuse and doctor them ; and , worst of all , no Mrs. Jo to make home life pleasant and life easy for them .<br>To their great affliction , Mrs. Jo seemed to consider herself one of the offended girls , for she hardly spoke to the outcasts , looked as if she did not see them when she passed , and was always too busy now to attend to their requests .",
"Claim": "Nan was there to doctor them.",
"Part of Claim in Background": "no Nan to amuse and doctor them",
"Evaluation": "In the claim, Nan is there to doctor them. In the background, it is stated that there is no Nan to doctor them. This is inconsistent."
"Verdict": "Contradiction"
},
{
"Background": "How to avoid eviction<br>Make timely payments. The easiest way to avoid eviction is to pay your rent on time. Read your lease to see if there is any grace period (such as three or five days).",
"Claim": "most leases have grace periods for payment",
"Part of Claim in Background": "if there is any grace period (such as three or five days)",
"Evaluation": "In the claim, most leases have grace periods for payment. In the background, it is stated that there may be a grace period of three or five days. They neither contradict nor entail each other so it's neutral."
"Vedict": "Neutral"
},
{
"Background": "At least 58 people are now dead as a result of the recent flooding in Yemen, and at least 20,000 in the country have no access to shelter. Five people are also reported missing. The Yemeni government has pledged to send tents to help the homeless. The flooding is caused by the recent heavy rain in Yemen, which came as a shock due to the fact that the country only receives several centimeters of rain per year.",
"Claim": "Rising water levels in Yemen has resulted in people passing away",
"Part of Claim in Background": "At least 58 people are now dead as a result of the recent flooding in Yemen",
"Evaluation": "In the claim, rising water levels in Yemen has resulted in people passing away. In the background, it is stated that at least 58 people are now dead as a result of the recent flooding in Yemen. This entails the claim.",
"Verdict": "Entailment"
},
{
"Background": "I wonder how he expects us to put forward a strong position on these issues if we are not part of the World Trade Organization or if we were not in a position, with meetings like Seattle, where we can pull together different countries from around the world that have a similar position to ours.",
"Claim": "there were never meetings in Seattle",
"Part of Claim in Background": "with meetings like Seattle",
"Evaluation": "In the claim, there were never meetings in Seattle. In the background, it is stated that there were meetings like Seattle. This is inconsistent.",
"Verdict": "Contradiction"
},
{
"Background": "Luciano Chailly (born Ferrara, January 19, 1920 – died Milan, December 24, 2002) was an Italian composer and arts administrator of French descent. He was the father of harpist Cecilia Chailly, conductor Riccardo Chailly and journalist Floriana Chailly. As a composer, Chailly was best known for his operas, many of which were composed to libretti by Dino Buzzati.",
"Claim": "Luciano Chailly was born in January",
"Part of Claim in Background": "January 19, 1920",
"Evaluation": "In the claim, Luciano Chailly was born in January. In the background, it is stated that he was born on January 19, 1920. This entails the claim.",
"Verdict": "Entailment"
},
{
"Background": "Oil Vials<br>I purchased several vials of perfumed oils. I picked all sweet scents. They were very realistic, and I was quite pleased. I put some on first thing in the morning. My husband complimented me, as did everyone in the office that day.",
"Claim": "The vials are large.",
"Part of Claim in Background": "None",
"Evaluation": "In the claim, the vials are large. In the background, there is no mention of the size of the vials. This is neutral.",
"Verdict": "Neutral"
},
{
"Background": "Segura had X-rays on his hand after getting hit by a pitch Friday against the Astros, Shannon Drayer of 710 ESPN Seattle reports. Segura was hit in the eighth inning but did not leave the game. He appears to have avoided a serious injury but is considered day-to-day.",
"Claim": "Segura was hit Friday.,
"Part of Claim in Background": "Friday against the Astros",
"Evaluation": "In the claim, Segura was hit Friday. In the background, it is stated that he was hit Friday against the Astros. This entails the claim.",
"Verdict": "Entailment"
},
{
"Background": "Grudge<br>Tom had a grudge against Ana. He decided he was going to slit her throat. Tom waited in the shadows by Ana's house. When he saw her red coat he lunged out, slicing and stabbing! Tom had only killed Ana's friend, who had borrowed her coat.",
"Claim": "Ana lost her coast",
"Part of Claim in Background": "Ana's friend, who had borrowed her coat",
"Evaluation": "In the claim, Ana lost her coast. In the background, it is stated that Ana's friend had borrowed her coat. This is inconsistent.",
"Verdict": "Contradiction"
},
{
"Background": "London Calling is the third studio album by English punk rock band the Clash. It was released as a double album in the United Kingdom on 14 December 1979 by Columbia Records, and in the United States in January 1980 by Epic Records. "London Calling" is an album that incorporates a range of styles, including punk, reggae, rockabilly, ska, New Orleans R&B, pop, lounge jazz, and hard rock.",
"Claim": "Thanks to the rockabilly and R&B the album was released in the United States in 1980.",
"Part of Claim in Background": "in the United States in January 1980 by Epic Records",
"Evaluation": "In the claim, the album was released in the United States in 1980. In the background, it is stated that it was released in the United States in January 1980 by Epic Records. They neither contradict nor entail each other so it's neutral."
"Verdict": "Neutral"
}
"""

## alright, not great, but let's see if this 9 shot works at all (not feeling great)

In [8]:
dev = pd.read_json("anli_v1.0/R3/dev.jsonl",lines=True)

In [9]:
dev.head()

,uid,context,hypothesis,label,model_label,emturk,genre,reason,tag
0,50a5d4bc-bb8b-44f9-ba07-95dfaf5536ab,one of the orders issued by Ochola in April Lo...,The decision to move the photocopier business ...,e,n,False,news,I made an obvious inference from the text that...,r3_dev
1,08789750-b5fe-478d-8d3f-06a01e3a8b1e,"If you can dream it, you can achieve it — unle...",The crowd believed they knew the name of the g...,e,n,False,news,"Because the crowd was chanting its name, the c...",r3_dev
2,a960c8cc-2f55-4269-a028-7f714479b068,"The Kremlin described as ""complete nonsense"" o...",The Kremlin used the word nonsense multiple ti...,e,n,False,news,I based my statement on supposed word count ba...,r3_dev
3,95e97907-0267-45ae-89fc-7e1fce39ee77,Police said that a 21-year-old man was discove...,The victim was less than a quarter century old,e,c,False,news,21 is less than 25. I think the system got it ...,r3_dev
4,5bd3435a-e52e-4688-9bc3-746b47c53469,Hurricane Harvey has caused devastating floods...,Houston is not the only area impacted by Hurri...,e,n,False,news,Louisiana was also impacted,r3_dev


In [14]:
results_dict = {}
correct = 0
done = 0

In [15]:

for row in dev.iterrows():
    if row[0] in results_dict:
        continue
    newPrompt = basePrompt + createTargetExample(row[1])
    res = query(newPrompt, stop="}", engine="text-davinci-001")
    results_dict[row[0]] = {
        "res": res,
        "label": row[1].label,
        "pred": evaluateAnswer(res)
    }
    done += 1
    if evaluateAnswer(res) == row[1].label:
        correct += 1
    
    if done % 10 == 0:
        with open("textdavinci001-naivefewshot.json", "w") as f:
            json.dump(results_dict, f)    
    
    print(f"Done {done} of {len(dev)}. Score: {correct}/{done} ({correct/done*100:.2f}%)")

with open("textdavinci001-naivefewshot.json", "w") as f:
    json.dump(results_dict, f)    


Done 1 of 1200. Score: 0/1 (0.00%)
Done 2 of 1200. Score: 1/2 (50.00%)
Done 3 of 1200. Score: 2/3 (66.67%)
Done 4 of 1200. Score: 2/4 (50.00%)
Done 5 of 1200. Score: 2/5 (40.00%)
Done 6 of 1200. Score: 3/6 (50.00%)
Done 7 of 1200. Score: 4/7 (57.14%)
Done 8 of 1200. Score: 4/8 (50.00%)
Done 9 of 1200. Score: 4/9 (44.44%)
Done 10 of 1200. Score: 4/10 (40.00%)
Done 11 of 1200. Score: 4/11 (36.36%)
Done 12 of 1200. Score: 4/12 (33.33%)
Done 13 of 1200. Score: 4/13 (30.77%)
Done 14 of 1200. Score: 4/14 (28.57%)
Done 15 of 1200. Score: 4/15 (26.67%)
Done 16 of 1200. Score: 5/16 (31.25%)
Done 17 of 1200. Score: 5/17 (29.41%)
Done 18 of 1200. Score: 6/18 (33.33%)
Done 19 of 1200. Score: 7/19 (36.84%)
Done 20 of 1200. Score: 7/20 (35.00%)
Done 21 of 1200. Score: 8/21 (38.10%)
Done 22 of 1200. Score: 9/22 (40.91%)
Done 23 of 1200. Score: 10/23 (43.48%)
Done 24 of 1200. Score: 10/24 (41.67%)
Done 25 of 1200. Score: 10/25 (40.00%)
Done 26 of 1200. Score: 10/26 (38.46%)
Done 27 of 1200. Score: 10/